# Convert measurement angles to k-space

In this example, we will use the NeXus format to automatically detect and read the angles from an ARPES measurement and do a k-space conversion with pyArpes.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np

from arpes.io import load_data
from arpes.endstations.plugin.nexus import NeXusEndstation
from arpes.utilities.conversion import convert_to_kspace


%matplotlib inline

# Download data

First we need to automatically download the data from [Nomad](https://nomad-lab.eu/prod/v1/api/v1/entries/xV9yMspIyXMfia6nNw3NIbnlZ91H/raw/Scan1496.nxs).

In [ ]:
! curl -o Scan1496.nxs https://nomad-lab.eu/prod/v1/api/v1/entries/xV9yMspIyXMfia6nNw3NIbnlZ91H/raw/Scan1496.nxs

## Load data

In [ ]:
Scan1496 = load_data("Scan1496.nxs", location=NeXusEndstation)

Apply rotational offset from experimental geometry

In [ ]:
Scan1496.S.apply_offsets({
    "chi": 19/180*np.pi,
})

Subtract photon energy

In [ ]:
Scan1496_shifted = Scan1496
Scan1496_shifted['eV'] = Scan1496_shifted['eV'] - Scan1496.attrs["hv"].magnitude

We read all metadata from the nexus file into xarray attributes for easy programmatic access.

In [ ]:
Scan1496_shifted.attrs

We can easily readout interesting values including their units, e.g., the incident photon energy of the beam:

In [ ]:
Scan1496_shifted.attrs['instrument/beam_probe/incident_energy']

## Generate Fermi surface

In [ ]:
fsmap = Scan1496_shifted.S.generic_fermi_surface(0)
fsmap.S.plot()

## Convert into k-space

In [ ]:
fsmap_converted = convert_to_kspace(
    fsmap,
    kx=np.linspace(-.6, .6, 400),
    ky=np.linspace(-1.2, 1.2, 400),
)

In [ ]:
fsmap_converted.T.plot()

In [10]:
Scan1496_converted = convert_to_kspace(
    Scan1496,
    kx=np.linspace(-.6, .6, 400),
    ky=np.linspace(-1.2, 1.2, 400),
)

In [ ]:
Scan1496_converted.S.generic_fermi_surface(0).T.plot()

In [ ]:
Scan1496.loc[{"eV":slice(1, 1.2)}].sum(dim="eV").S.plot()

In [ ]:
Scan1496_converted.loc[{"eV":slice(1, 1.2)}].sum(dim="eV").S.plot()